# RSAisEasy (Writeup) [ESP]
###### _Categoría: Crypto_  
###### _Dificultad: Fácil_
###### _by: D-Cryp7_

Descripción: 
* I think this is safe... Right?

Bibliotecas externas a utilizar:

In [ ]:
! pip install pycryptodome
! pip install factordb-python

Se nos entrega el siguiente código



```python
from Crypto.Util.number import bytes_to_long, getPrime
from secrets import flag1, flag2
from os import urandom

flag1 = bytes_to_long(flag1)
flag2 = bytes_to_long(flag2)

p, q, z = [getPrime(512) for i in range(3)]

e = 0x10001

n1 = p * q
n2 = q * z

c1 = pow(flag1, e, n1)
c2 = pow(flag2, e, n2)

E = bytes_to_long(urandom(69))

print(f'n1: {n1}')
print(f'c1: {c1}')
print(f'c2: {c2}')
print(f'(n1 * E) + n2: {n1 * E + n2}')
```



con sus respectivas variables:

In [2]:
n1 = 101302608234750530215072272904674037076286246679691423280860345380727387460347553585319149306846617895151397345134725469568034944362725840889803514170441153452816738520513986621545456486260186057658467757935510362350710672577390455772286945685838373154626020209228183673388592030449624410459900543470481715269
c1 = 92506893588979548794790672542461288412902813248116064711808481112865246689691740816363092933206841082369015763989265012104504500670878633324061404374817814507356553697459987468562146726510492528932139036063681327547916073034377647100888763559498314765496171327071015998871821569774481702484239056959316014064
c2 = 46096854429474193473315622000700040188659289972305530955007054362815555622172000229584906225161285873027049199121215251038480738839915061587734141659589689176363962259066462128434796823277974789556411556028716349578708536050061871052948425521408788256153194537438422533790942307426802114531079426322801866673
x = 601613204734044874510382122719388369424704454445440856955212747733856646787417730534645761871794607755794569926160226856377491672497901427125762773794612714954548970049734347216746397532291215057264241745928752782099454036635249993278807842576939476615587990343335792606509594080976599605315657632227121700808996847129758656266941422227113386647519604149159248887809688029519252391934671647670787874483702292498358573950359909165677642135389614863992438265717898239252246163

 Entonces, $n_1$ y $n_2$ son números de 1024 bits (aproximadamente), veamos si podemos factorizar $n_1$ con [factordb](http://factordb.com) para romper RSA sólo calculando la llave privada $d$

In [3]:
from factordb.factordb import FactorDB
n1_factor = FactorDB(n1)
if (n1_factor.connect().status_code == 200):
  n1_factor = n1_factor.get_factor_list()
p, q = n1_factor[0], n1_factor[1]
p, q, p * q == n1

(8413387656561188778435613942028835678781206299389177514340760123063579360223360470566083306606450007991287094526418200038784207648097820069671213638771543,
 12040644312371555810530782070969893153760288255333349208608058511112776958879208815174991008199408527954332776642365069284747758115478414463195873149420483,
 True)

 sí podemos! así que la `flag1` puede ser obtenida a través de la siguiente función

In [6]:
from Crypto.Util.number import inverse, long_to_bytes
import codecs
def rsa_decrypt(n, p, q, e, c):
    phi = (p - 1) * (q - 1)
    d = inverse(e, phi)
    m = pow(c, d, n)
    print("m :", m)
    m_decode = long_to_bytes(m)
    return m_decode

In [7]:
e = 0x10001
flag1 = rsa_decrypt(n1, p, q, e, c1)
flag1

m : 6927755227520641588229085684836623901969480804170150756


b'HTB{1_m1ght_h4v3_m3ss3d'

Para la `flag2`, necesitamos calcular $n_2$ primero, pero lo único que tenemos es una combinación lineal entre $n_1$ y $n_2$. No importa! esto es suficiente para hacer un truquito matemático :D
\begin{equation}
  n_1 \cdot E + n_2 = x
\end{equation}
si aplicamos mod $n_1$ a ambos lados de la ecuación
\begin{equation}
  n_2 \equiv x \text{ mod } n_1
\end{equation}
obtenemos la siguiente congruencia, lo cual nos deja un valor posible de $n_2$. El siguiente código lo calcula

In [8]:
n2 = x % n1
n2

100136903041423020991425823526737746365573197640035952973693624809721624428963253203282593974533722584391447008912397042291986993273828302711324440847902763039627790146764630023926517236880457533976468679976683705170312329736955922713306570804595070537102421450884645497775455984735279182873866159334387494837

así, podemos obtener la `flag2` directamente

In [9]:
z = n2 // p
flag2 = rsa_decrypt(n2, p, z, e, c2)
flag2

m : 9143084635788955941405366124572245421572627941723225981


b'_uP_jU$t_4_l1ttle_b1t?}'

Finalmente, concatenamos la `flag1` con `flag2` ;)

In [10]:
flag = flag1 + flag2
flag

b'HTB{1_m1ght_h4v3_m3ss3d_uP_jU$t_4_l1ttle_b1t?}'